In [ ]:
!pip install readlif python-bioformats tifffile imageio scikit-image

6179,6046 gives error because they match but dont have the same z stack depth

5878 has another subject (musc), some has vaso2 others have vaso 2

4092 which is MSA is also stored in a control image dont know why, be carefull to remove it manually

## version producing 3channel rgb images

In [6]:
%pip install readlif

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# new version to save also png images
from readlif.reader import LifFile
from IPython.display import display
import pandas as pd
import numpy as np
import os
import re
import tifffile
import numpy as np
#import cv2
import matplotlib.pyplot as plt


def parse_name(name):
    # print(f"Parsing name: {name}") #uncomment to see all names that are parsed
    id_match = re.search(r"(\d{4})", name)
    subject_match = re.search(r"\b(gh(?:\s*n?\s*\d)?)\b", name, re.IGNORECASE) #recognize gh, gh 2, gh x, gh n x, ghnx
    # recgonize gh2, gh 2, gh
    if id_match and subject_match:
        id_ = id_match.group(1)
        subject = subject_match.group(1).lower()
        # print(f"Extracted ID: {id_}, Subject: {subject}") #uncomment to see all the IDs and Subjects extracted
        return id_, subject
    # print("ID or Subject not found") #uncomment to see all the names that don't match the criteria
    return None, None

def normalize_channel(channel_data):
    """
    performs a min-max normalization of the channel_data to the 0-255 range and converts it to uint8
    """
    min_val = np.min(channel_data)
    max_val = np.max(channel_data)
    
    if max_val - min_val == 0:
        # Handle cases where the channel has a constant value
        normalized_channel = np.zeros_like(channel_data, dtype=np.uint8)
    else:
        normalized_channel = ((channel_data - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    
    return normalized_channel

def create_rgb_projection(image_data, method="max", red_channel=3, green_channel=0, blue_channel=1):
    """
    Creates a 3-channel RGB projection from a multi-channel image.
    without channel normalization""

    Args:
        image_data: NumPy array with shape (Z, Y, X, C) or (C, Y, X) where C is the number of channels.
        method: Projection method ("max" or "mean").
        red_channel: Index of the channel to use for red.
        green_channel: Index of the channel to use for green.
        blue_channel: Index of the channel to use for blue.

    Returns:
        NumPy array with shape (Y, X, 3) representing the RGB projection.
    """
    print(f"input shape before rgb: {image_data.shape}") # (50, 1024, 1024, 4)
    if image_data.ndim == 4:  # (Z, Y, X, C)
        if method == "max":
            projection_func = np.max
        elif method == "mean":
            projection_func = np.mean
        else:
            raise ValueError("Invalid projection method. Choose 'max' or 'mean'.")

        print(f"Creating RGB projection using {method} projection method")
        print(f"image shape: {image_data.shape}")
        # Ensure the selected channels are within the valid range
        num_channels = image_data.shape[3]
        if not all(0 <= c < num_channels for c in [red_channel, green_channel, blue_channel]):
            raise ValueError(f"Invalid channel indices. Image has {num_channels} channels.")

        # Project along the Z-axis for each selected channel
        red_proj = projection_func(image_data[:, :, :, red_channel], axis=0)
        green_proj = projection_func(image_data[:, :, :, green_channel], axis=0)
        blue_proj = projection_func(image_data[:, :, :, blue_channel], axis=0)
        rgb_image = np.stack([red_proj, green_proj, blue_proj], axis=-1) # (Y, X, 3)
    elif image_data.ndim == 3:  # (C, Y, X)
        # No projection needed, just select channels
        num_channels = image_data.shape[0]
        if not all(0 <= c < num_channels for c in [red_channel, green_channel, blue_channel]):
            raise ValueError(f"Invalid channel indices. Image has {num_channels} channels.")
        red_proj = image_data[red_channel, :, :]
        green_proj = image_data[green_channel, :, :]
        blue_proj = image_data[blue_channel, :, :]
        rgb_image = np.transpose(
            np.stack([
                image_data[red_channel],
                image_data[green_channel],
                image_data[blue_channel]
            ]),
            (1, 2, 0)  # From (3, Y, X) to (Y, X, 3)
        )
    else:
        raise ValueError("Invalid image_data shape. Must be either (Z, Y, X, C) or (C, Y, X).")

    # Normalize each channel to 0-255 and convert to uint8
    #red_proj = normalize_channel(red_proj)
    #green_proj = normalize_channel(green_proj)
    #blue_proj = normalize_channel(blue_proj)

    # Stack the channels to create an RGB image
    
    print(f"output shape after rgb projection, should be (X,Y,3): {rgb_image.shape}") #(1024, 1024, 3)

    return rgb_image

#!TODO put dapi overlay to false
def save_colored_mip_png(merged_data, output_path, id_, subject, red_channel=0, green_channel=1, blue_channel=2, dapi_overlay=True):
    """
    Create and save colored Maximum Intensity Projection (MIP) png.
    
    Args:
        merged_data: numpy array with shape (Z, C, Y, X) - Z-stack of multi-channel image data
        output_path: str - base directory for saving output
        id_: str - sample identifier
        subject: str - subject type (e.g., 'MSA', 'PD')
        red_channel: int - index of channel to use for red (default=0)
        green_channel: int - index of channel to use for green (default=1)
        blue_channel: int - index of channel to use for blue (default=2)
        dapi_overlay: bool - whether to overlay DAPI channel (assumed to be channel 3)
    
    Returns:
        str: Path to the saved MIP image
    """
    # Create output directory for colored MIPs if it doesn't exist
    mip_folder = os.path.join(output_path, "Colored_MIPs")
    if not os.path.exists(mip_folder):
        os.makedirs(mip_folder)

    # Get number of channels and validate channel indices
    num_channels = merged_data.shape[1]
    rgb_channels = [red_channel, green_channel, blue_channel]
    if not all(0 <= c < num_channels for c in rgb_channels):
        raise ValueError(f"Invalid channel indices for RGB. Image has {num_channels} channels.")
    if dapi_overlay and num_channels < 4:
        print("Warning: DAPI overlay requested but image has fewer than 4 channels. Skipping DAPI overlay.")
        dapi_overlay = False

    # Create maximum intensity projection for each channel
    mip_channels = [np.max(merged_data[:, c, :, :], axis=0) for c in range(num_channels)]

    # Normalize RGB channels and create composite image
    normalized_channels = [normalize_channel(mip_channels[c]) for c in rgb_channels]
    mip_image = np.zeros((*mip_channels[0].shape, 3), dtype=np.uint8)
    for i, c in enumerate(rgb_channels):
        mip_image[:, :, i] = normalized_channels[i]

    # Add DAPI channel overlay if requested
    if dapi_overlay:
        dapi = normalize_channel(mip_channels[3])
        for i in range(3):
            mip_image[:, :, i] = np.maximum(mip_image[:, :, i], dapi)

    # Save the colored MIP as PNG
    mip_folder = os.path.join(mip_folder, "colored_mips_PNG")
    if not os.path.exists(mip_folder):
        os.makedirs(mip_folder)
    mip_path = os.path.join(mip_folder, f"{id_}_{subject}_colored_mip.png")
    plt.imsave(mip_path, mip_image)
    print(f"Saved colored MIP as {mip_path}")
    return mip_path

def trim_to_match_z(data_TH, data_DAPI):
    """
    Trim Z-stacks of TH and DAPI images to match the smaller Z dimension.
    
    Args:
        data_TH: NumPy array of TH image data with shape (Z, Y, X, C)
        data_DAPI: NumPy array of DAPI image data with shape (Z, Y, X, C)
        
    Returns:
        Tuple of (trimmed_TH, trimmed_DAPI) where both arrays have matching Z dimensions
        equal to the minimum Z depth between the inputs
    """
    min_z = min(data_TH.shape[0], data_DAPI.shape[0])
    data_TH_trimmed = data_TH[:min_z, :, :, :]
    data_DAPI_trimmed = data_DAPI[:min_z, :, :, :]
    return data_TH_trimmed, data_DAPI_trimmed

def load_lif_images(lif_path):
    """
    Load images from a LIF file and extract image data.
    
    Args:
        lif_path: str - path to the LIF file
        
    Returns:
        list of dicts, where each dict contains:
            'name': str - name of the image
            'data': numpy array with shape (Z, Y, X, C) where:
                Z is number of z-planes
                Y is height
                X is width 
                C is number of channels
    """
    lif = LifFile(lif_path)
    images = []
    
    for img in lif.get_iter_image():
        num_channels = img.channels
        z_depth = img.dims.z
        frames = []
        print(f"Image name: {img.name}, Channels: {num_channels}, Z-depth: {z_depth}")
        
        # Create array to store Z-stack data
        z_stack_data = []
        
        # Iterate through Z-planes
        for z in range(z_depth):
            channels_data = []
            # Get each channel for this Z-plane
            for c in range(num_channels):
                frame = img.get_frame(z=z, c=c)
                frame_array = np.array(frame)
                channels_data.append(frame_array)
            # Stack channels for this Z-plane
            z_plane = np.stack(channels_data, axis=-1)
            z_stack_data.append(z_plane)
        
        # Stack all Z-planes
        data_array = np.stack(z_stack_data, axis=0)  # Shape: (Z, Y, X, C)
        images.append({'name': img.name, 'data': data_array})
        
    return images

def group_images_by_id_subject(images):
        """
        Groups images by their ID and subject into a dictionary.
        
        Args:
            images: List of dictionaries containing image data and names
            
        Returns:
            Dictionary with (id, subject) tuples as keys and lists of images as values
            { 
                (id1, subject1): [img1, img2, ...],
                (id2, subject2): [img3, img4, ...],
                ...}
        """
        series_dict = {}
        for img in images:
            name = img['name']
            id_, subject = parse_name(name)
            if id_ and subject:
                key = (id_, subject)
                if key not in series_dict:
                    series_dict[key] = []
                series_dict[key].append(img)
        return series_dict

def visualize_dict_with_pandas(series_dict):
    """
    Visualize the content of series_dict in a tabular format using pandas.

    Parameters:
    - series_dict: dict
        A dictionary where keys are tuples (ID, subject),
        and values are lists of dictionaries containing 'name' and 'data'.

    Returns:
    - A pandas DataFrame with the content of the dictionary.
    """
    # Prepare data for the DataFrame
    rows = []
    for (ID, subject), images in series_dict.items():
        for image in images:
            rows.append({
                "ID": ID,
                "Subject": subject,
                "Image Name": image.get("name"),
                "Data Shape": image.get("data", np.array([])).shape  # Show shape of the image array
            })
    
    # Create a DataFrame
    df = pd.DataFrame(rows)
    return df

def visualize_mips(data, title):
                n_channels = data.shape[3] #=4
                print(f"N channels: {n_channels}")
                print(f"Data shape: {data.shape}")
                
                # Create MIPs with normalization for each channel
                mips = []
                for c in range(n_channels):
                    channel_data = data[:,:,:,c]
                    mip = np.max(channel_data, axis=0)
                    # Normalize each MIP independently
                    mip = normalize_channel(mip)
                    mips.append(mip)
                
                # Create the figure
                fig, axes = plt.subplots(1, n_channels, figsize=(4*n_channels, 4))
                if n_channels == 1:
                    axes = [axes]
                
                # Display each channel
                for i, (mip, ax) in enumerate(zip(mips, axes)):
                    ax.imshow(mip, cmap='gray', vmin=0, vmax=255)
                    ax.set_title(f'{title} Channel {i}')
                    ax.axis('off')
                
                plt.tight_layout()
                plt.show()
                
                # Print min/max values for debugging
                for i, mip in enumerate(mips):
                    print(f"Channel {i} - Min: {np.min(mip)}, Max: {np.max(mip)}")
                
def visualize_single_channel(img_dict, channel=2, title=None, color = 'gray'):
    """
    Visualize a single channel from a dictionary containing image data.
    
    Args:
        img_dict: Dictionary containing:
            - 'name': str, name of the image
            - 'data': numpy.ndarray with shape (Z, Y, X, C)
        channel: int, index of channel to visualize (default=1)
        title: str, custom title for the plot (default=None)
    """
    if not isinstance(img_dict, dict) or 'data' not in img_dict:
        raise ValueError("Input must be a dictionary with 'data' key")
    
    data = img_dict['data']
    if data.ndim != 4:
        raise ValueError(f"Expected 4D data (Z,Y,X,C), got shape {data.shape}")
    
    if channel >= data.shape[3]:
        raise ValueError(f"Channel index out of range. Image has {data.shape[3]} channels")
    
    # Create maximum intensity projection for selected channel
    mip = np.max(data[:, :, :, channel], axis=0)
    
    # Normalize the projection
    mip_norm = normalize_channel(mip)
    
    # Set default title if none provided
    if title is None:
        title = f'{img_dict["name"]}\nChannel {channel}'
    
    # Create the visualization
    plt.figure(figsize=(8, 8))
    im = plt.imshow(mip_norm, cmap=color)
    plt.title(f'{title}\nmin={mip.min():.2f}, max={mip.max():.2f}')
    plt.axis('off')
    plt.colorbar(im)
    plt.show()

# Example usage:
# visualize_single_channel(img_DAPI, channel=2, title='DAPI Channel')

# NB:we assume that the channel color order is consistent with each image green,blue,grey,red
# if the order is different, the function will not work properly
import os
import numpy as np
import matplotlib.pyplot as plt

def process_pairs(lif_path, channel_to_add_from_DAPI=2, output_dir="merged_results", rgb_output_dir="rgb_projections"):
    # Create output directories if they don't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(rgb_output_dir):
        os.makedirs(rgb_output_dir)

    images_dict = load_lif_images(lif_path)  # List of dicts: [{'name': ..., 'data': ...}, ...]
    series_dict = group_images_by_id_subject(images_dict)  # Keys: (id, subject) and values: list of images

    for key, images in series_dict.items():
        print(f"Found {len(images)} images for ID: {key[0]}, Subject: {key[1]}")
    df = visualize_dict_with_pandas(series_dict)
    display(df)

    for (id_, subject), imgs in series_dict.items():
        print(f"\nProcessing ID: {id_}, Subject: {subject}")
        # Define patterns for matching image names
        dapi_patterns = ['dapi']
        color_patterns = ['grey', 'gray']
        th_patterns = ['th']
        b_patterns = ['blue']

        # Find DAPI image (using name matching for 'dapi' or 'grey/gray')
        img_DAPI = next(
            (img for img in imgs 
             if any(d in img['name'].lower() for d in dapi_patterns) 
             or any(c in img['name'].lower() for c in color_patterns)), 
            None
        )

        # Find TH image (using name matching for 'th' or 'blue')
        img_TH = next(
            (img for img in imgs 
             if any(t in img['name'].lower() for t in th_patterns)
             or any(b in img['name'].lower() for b in b_patterns)), 
            None
        )

        print(f"DAPI image name found: {img_DAPI['name']}" if img_DAPI else "No DAPI image found")
        print(f"TH   image name found: {img_TH['name']}" if img_TH else "No TH image found")
        if img_DAPI and img_TH:
            print("names are equal?", img_DAPI['name'] == img_TH['name'])
        
        same_image = False
        if img_DAPI and img_TH and img_DAPI['name'] == img_TH['name']:
            print("The same image has both DAPI and TH channels")
            same_image = True

        if img_DAPI and img_TH:  # if both images are found
            data_DAPI = img_DAPI['data']  # shape (Z, Y, X, C)
            data_TH = img_TH['data']
            print(f"Data DAPI shape: {data_DAPI.shape}, Data TH shape: {data_TH.shape}")

            if not same_image:
                visualize_single_channel(img_DAPI, channel=2, title='DAPI Channel', color='gray')
                print("\nTH image channels:")
                visualize_single_channel(img_TH, channel=2, title='TH Channel', color='Blues')
            else:
                print("Image channels:")
                visualize_single_channel(img_DAPI, channel=2, title='DAPI Channel', color='gray')
            
            # If both images already have 4 channels, choose one to process
            if data_DAPI.shape[3] == 4 and data_TH.shape[3] == 4:
                data_to_process = data_DAPI if data_DAPI.shape[0] >= data_TH.shape[0] else data_TH
                save_images(data_to_process, output_dir, rgb_output_dir, id_, subject)
            else:
                # Ensure the number of Z-slices match
                if data_DAPI.shape[0] != data_TH.shape[0]:
                    data_TH, data_DAPI = trim_to_match_z(data_TH, data_DAPI)
                
                # Instead of using a fixed channel index, display a MIP for each channel of the DAPI image
                num_channels = data_DAPI.shape[3]
                print("Displaying maximum intensity projections (MIPs) for each DAPI channel:")
                for ch in range(num_channels):
                    # Compute the maximum intensity projection over the Z-axis for this channel
                    mip = data_DAPI[..., ch].max(axis=0)  # shape (Y, X)
                    plt.figure()
                    plt.imshow(mip, cmap='gray')
                    plt.title(f"DAPI Image - Channel {ch} MIP")
                    plt.axis('off')
                    plt.show()
                
                # Ask the user to choose which channel to use for merging
                try:
                    selected_channel = int(input("Enter the channel index to use for merging from the DAPI image: "))
                except Exception as e:
                    print("Invalid input. Skipping merging for this pair.")
                    continue

                if selected_channel < 0 or selected_channel >= num_channels:
                    print("Selected channel index is out of range. Skipping merging for this pair.")
                    continue

                # Extract the selected channel and expand dimensions so it can be concatenated along the channel axis
                dapi_channel = data_DAPI[..., selected_channel]
                dapi_channel = np.expand_dims(dapi_channel, axis=3)

                try:
                    # Merge the TH image with the selected DAPI channel (along the channel axis)
                    merged = np.concatenate((data_TH, dapi_channel), axis=3)
                    save_images(merged, output_dir, rgb_output_dir, id_, subject)
                except ValueError as ve:
                    print(f"Failed to concatenate arrays: {ve}")
                    continue
        else:
            # If no matching DAPI or TH images are found, try to locate a 4-channel image
            print("No DAPI or TH images found. Searching for a 4-channel image...")
            four_channel_img = next((img for img in imgs if img['data'].shape[3] == 4), None)
            if four_channel_img:
                print(f"Found 4-channel image for ID: {id_}, Subject: {subject}")
                data = four_channel_img['data']
                save_images(data, output_dir, rgb_output_dir, id_, subject)
            else:
                print(f"Skipping {id_}, {subject}: No matching images and the only image found has less than 4 channels")


def save_images(data, output_dir, rgb_output_dir, id_, subject):
    """Save the merged data as TIFF, produce a 4-channel MIP TIFF, save colored MIP, and save RGB projection."""
    # Save merged data as 4c no projection and no normalization TIFF
    out_path = os.path.join(output_dir, f"{id_}_{subject}_merged.tif")
    try:
        data_reoriented = np.moveaxis(data, 3, 1) # Move the 3rd axis (channels) to the 2nd position 
        print(f"Data reoriented shape: {data_reoriented.shape}") # (Z, C, Y, X)=(30, 4, 1024, 1024)
        print(f"Saving {data_reoriented.shape[1]} channel tiff to {out_path}")
        tifffile.imwrite(
            out_path,
            data_reoriented,
            imagej=True,
            metadata={
                'axes': 'ZCYX',
                'Channels': data_reoriented.shape[1],
                #'Channel_colors': ['red', 'green', 'blue', 'gray']
                'Channel_colors': ['green', 'blue', 'gray', 'red']
            }
        )
        print(f"Saved {out_path} with shape {data_reoriented.shape}")

        # Create and save 4-channel normalized MIP
        mip_data = np.max(data_reoriented, axis=0) # Maximum intensity projection along Z-axis
        print(f"MIP data shape: {mip_data.shape}") # (C, Y, X)=(4, 1024, 1024)
        normalized_mip_channels = [normalize_channel(mip_data[c]) for c in range(mip_data.shape[0])]
        normalized_mip = np.stack(normalized_mip_channels, axis=0)
        print(f"channel Normalized MIP stack shape: {normalized_mip.shape}") # (C, Y, X)=(4, 1024, 1024)
        mip_out_path = os.path.join(output_dir,"channel_Normalized_MIPS", f"{id_}_{subject}_mip.tif")
        os.makedirs(os.path.dirname(mip_out_path), exist_ok=True) 
        tifffile.imwrite(
            mip_out_path,
            normalized_mip,
            imagej=True,
            metadata={'axes': 'CYX', 'Channels': normalized_mip.shape[0]}
        )
        print(f"Saved 4-channel Normalized MIP as {mip_out_path}")

        # Save colored MIP for visualization
        save_colored_mip_png(data_reoriented, output_dir, id_, subject)

        # Create and save RGB projection for pre-trained network input

        rgb_projection = create_rgb_projection(data, method="max")
        print(f"RGB projection shape: {rgb_projection.shape}") # (Y, X, 3)=(1024, 1024, 3)
        rgb_projection = np.moveaxis(rgb_projection, 2, 0)  # Move the 3rd axis (channels) to the 1st position
        rgb_out_path = os.path.join(rgb_output_dir, f"{id_}_{subject}_rgb.tif")
        print(f"RGB projection shape after moveaxis: {rgb_projection.shape}") # (Y, X, 3)=(1024, 1024, 3)
        # Modified code
        tifffile.imwrite(
            rgb_out_path,
            rgb_projection, # (Y, X, 3)
            imagej=True,
            metadata={'axes': 'CYX', 'Channels': rgb_projection.shape[0]}
        )    
        print(f"Saved RGB projection: {rgb_out_path}")

    except Exception as e:
        print(f"Failed to save: {e}")


msa_lif_ids = [
    4092, 4121, 5349, 5358, 5435, 5463,
    5717, 5745, 5753, 5767, 5776, 5878,
    5881, 5904, 5954, 5969, 5978, 5992,
    5996, 6046, 6050, 6053, 6060, 6085,
    6179
]

ctrl_lif_ids = [
    4115, 5167, 5168, 5197, 5199,
    5888, 6731, 6743
]

pd_lif_ids = [
    6008, 6320, 6323, 6337, 6340, 6351, 6366,
    6459, 6577, 6616, 6690, 6696, 6773
    ]

# from utils.directory_functions import *
# base_directory = get_base_directory()
# base_path = os.path.join(base_directory, "MIND_TESI_IMMAGINI")
base_path = r"C:\Users\lucaz\OneDrive - Politecnico di Milano\Notebooks\università\MAGISTRALE\TESI\MIND_TESI_IMMAGINI"
# Process MSA and Control files
for lif_ids, folder_name in [(msa_lif_ids, "MSA"), (ctrl_lif_ids, "controllo")]:
    print(f"\nProcessing LIF IDs: {lif_ids}")
    tiff4c_folder = os.path.join(base_path, "MANUAL_4c_tif")
    tiff3c_rgb_folder = os.path.join(base_path, "MANUAL_3cRGB_new")
    # Determine output directory based on which list we're processing
    if lif_ids == msa_lif_ids:
        output_dir = os.path.join(tiff4c_folder, "MSA")
        rgb_output_dir = os.path.join(tiff3c_rgb_folder, "MSA")
        source_folder = "MSA"
    else:
        output_dir = os.path.join(tiff4c_folder, "control")
        rgb_output_dir = os.path.join(tiff3c_rgb_folder, "control")
        source_folder = "controllo"  

    print(f"\nProcessing {source_folder} files...")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(rgb_output_dir, exist_ok=True)

    # Create source directory path
    source_dir = os.path.join(base_path, source_folder)
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"Source directory not found: {source_dir}")

    for lif_id in lif_ids:
        lif_path = os.path.join(source_dir, f"{lif_id}.lif")
        if not os.path.exists(lif_path):
            raise FileNotFoundError(f"LIF file not found: {lif_path}")

        process_pairs(
            lif_path,
            channel_to_add_from_DAPI=2,
            output_dir=output_dir,
            rgb_output_dir=rgb_output_dir
        )


Processing LIF IDs: [4092, 4121, 5349, 5358, 5435, 5463, 5717, 5745, 5753, 5767, 5776, 5878, 5881, 5904, 5954, 5969, 5978, 5992, 5996, 6046, 6050, 6053, 6060, 6085, 6179]

Processing MSA files...

Processing LIF IDs: [4115, 5167, 5168, 5197, 5199, 5888, 6731, 6743, 7141]

Processing controllo files...


5197 gh DL VIP r TH b Sinapto gr 63x z2 pinhole 06 z 022 , Channels: 4, Z-depth: 14 but it has 4 channels!! instead of expected 3

In [ ]:
# !TODO a function which saves 3channel tifs and make you select the channel to remove by making you visualize the mip of each channel
# something is wrong in this code since it shows images very differently from how they appears on fiji and save a merged results which shows
# a z stack instead of channels mip

import os
import numpy as np
import matplotlib.pyplot as plt

def process_pairs(lif_path, output_dir="merged_results", rgb_output_dir="rgb_projections"):
    # Create output directories if they don't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(rgb_output_dir):
        os.makedirs(rgb_output_dir)

    # Load and group images (assumed functions provided elsewhere)
    images_dict = load_lif_images(lif_path)  # [{'name': ..., 'data': ...}, ...]
    series_dict = group_images_by_id_subject(images_dict)  # {(id, subject): [img1, img2, ...], ...}
    
    for key, images in series_dict.items():
        print(f"Found {len(images)} images for ID: {key[0]}, Subject: {key[1]}")
    df = visualize_dict_with_pandas(series_dict)
    display(df)

    for (id_, subject), imgs in series_dict.items():
        print(f"\nProcessing ID: {id_}, Subject: {subject}")
        
        # Define patterns for matching DAPI and TH images
        dapi_patterns = ['dapi']
        color_patterns = ['grey', 'gray']
        th_patterns = ['th']
        b_patterns = ['blue']

        # Find a DAPI image (using name matching)
        img_DAPI = next(
            (img for img in imgs 
             if any(d in img['name'].lower() for d in dapi_patterns) 
             or any(c in img['name'].lower() for c in color_patterns)),
            None
        )
        
        # Find a TH image (using name matching)
        img_TH = next(
            (img for img in imgs 
             if any(t in img['name'].lower() for t in th_patterns)
             or any(b in img['name'].lower() for b in b_patterns)),
            None
        )

        if img_DAPI:
            print(f"DAPI image found: {img_DAPI['name']}")
        else:
            print("No DAPI image found")
        if img_TH:
            print(f"TH image found: {img_TH['name']}")
        else:
            print("No TH image found")
        
        # If a TH image is detected, use it.
        if img_TH:
            data_TH = img_TH['data']  # Expected shape: (Z, Y, X, C)
            print(f"TH image shape: {data_TH.shape}")

            # If the TH image already has 3 channels, simply save it.
            if data_TH.shape[3] == 3:
                print("TH image has 3 channels. Saving this image (ignoring the DAPI image).")
                save_images(data_TH, output_dir, rgb_output_dir, id_, subject)
            # If the TH image has 4 channels, let the user choose which channel to remove.
            elif data_TH.shape[3] == 4:
                num_channels = data_TH.shape[3]
                print("The TH image has 4 channels. Please inspect the following normalized MIPs and select the channel index to remove:")
                for ch in range(num_channels):
                    # Compute the maximum intensity projection (MIP) over the Z-axis for channel ch
                    mip = data_TH[..., ch].max(axis=0)  # shape (Y, X)
                    # Normalize the MIP for display
                    mip_norm = (mip - np.min(mip)) / (np.ptp(mip) + 1e-8)
                    plt.figure()
                    plt.imshow(mip_norm, cmap='gray')
                    plt.title(f"TH Image - Channel {ch} MIP")
                    plt.axis('off')
                    plt.show()
                try:
                    channel_to_delete = int(input("Enter the channel index to remove: "))
                except Exception as e:
                    print("Invalid input. Skipping this image.")
                    continue
                if channel_to_delete < 0 or channel_to_delete >= num_channels:
                    print("Selected channel index is out of range. Skipping this image.")
                    continue
                channels_to_keep = [i for i in range(num_channels) if i != channel_to_delete]
                data_final = data_TH[..., channels_to_keep]
                print(f"Saving image after removing channel {channel_to_delete}. New shape: {data_final.shape}")
                save_images(data_final, output_dir, rgb_output_dir, id_, subject)
            else:
                print(f"Unexpected number of channels in TH image: {data_TH.shape[3]}")
        else:
            # If no TH image is found, search for a 4-channel image in the series
            print("No TH image found. Searching for a 4-channel image...")
            four_channel_img = next((img for img in imgs if img['data'].shape[3] == 4), None)
            if four_channel_img:
                data = four_channel_img['data']
                print(f"Found 4-channel image: {four_channel_img['name']}.")
                num_channels = data.shape[3]
                print("Please inspect the following normalized MIPs and select the channel index to remove:")
                for ch in range(num_channels):
                    mip = data[..., ch].max(axis=0)
                    mip_norm = (mip - np.min(mip)) / (np.ptp(mip) + 1e-8)
                    plt.figure()
                    plt.imshow(mip_norm, cmap='gray')
                    plt.title(f"Image {four_channel_img['name']} - Channel {ch} MIP")
                    plt.axis('off')
                    plt.show()
                try:
                    channel_to_delete = int(input("Enter the channel index to remove: "))
                except Exception as e:
                    print("Invalid input. Skipping this image.")
                    continue
                if channel_to_delete < 0 or channel_to_delete >= num_channels:
                    print("Selected channel index is out of range. Skipping this image.")
                    continue
                channels_to_keep = [i for i in range(num_channels) if i != channel_to_delete]
                data_final = data[..., channels_to_keep]
                print(f"Saving image after removing channel {channel_to_delete}. New shape: {data_final.shape}")
                save_images(data_final, output_dir, rgb_output_dir, id_, subject)
            else:
                print(f"Skipping {id_}, {subject}: No matching images found.")


                
msa_lif_ids = [
    4092, 4121, 5349, 5358, 5435, 5463,
    5717, 5745, 5753, 5767, 5776, 5878,
    5881, 5904, 5954, 5969, 5978, 5992,
    5996, 6046, 6050, 6053, 6060, 6085,
    6179
]

ctrl_lif_ids = [
    4115, 5167, 5168, 5197, 5199,
    5888, 6731, 6743, 7141
]

pd_lif_ids = [
    6008, 6320, 6323, 6337, 6340, 6351, 6366,
    6459, 6577, 6616, 6690, 6696, 6773
    ]

# from utils.directory_functions import *
# base_directory = get_base_directory()
# base_path = os.path.join(base_directory, "MIND_TESI_IMMAGINI")
base_path = r"C:\Users\lucaz\OneDrive - Politecnico di Milano\Notebooks\università\MAGISTRALE\TESI\MIND_TESI_IMMAGINI"
# Process MSA and Control files
for lif_ids, folder_name in [(msa_lif_ids, "MSA"), (ctrl_lif_ids, "controllo")]:
    print(f"\nProcessing LIF IDs: {lif_ids}")
    tiff4c_folder = os.path.join(base_path, "MANUAL_3c_tif")
    tiff3c_rgb_folder = os.path.join(base_path, "MANUAL_3c_3cRGB_new")
    # Determine output directory based on which list we're processing
    if lif_ids == msa_lif_ids:
        output_dir = os.path.join(tiff4c_folder, "MSA")
        rgb_output_dir = os.path.join(tiff3c_rgb_folder, "MSA")
        source_folder = "MSA"
    else:
        output_dir = os.path.join(tiff4c_folder, "control")
        rgb_output_dir = os.path.join(tiff3c_rgb_folder, "control")
        source_folder = "controllo"  

    print(f"\nProcessing {source_folder} files...")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(rgb_output_dir, exist_ok=True)

    # Create source directory path
    source_dir = os.path.join(base_path, source_folder)
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"Source directory not found: {source_dir}")

    for lif_id in lif_ids:
        lif_path = os.path.join(source_dir, f"{lif_id}.lif")
        if not os.path.exists(lif_path):
            raise FileNotFoundError(f"LIF file not found: {lif_path}")

        process_pairs(
            lif_path,
            #channel_to_add_from_DAPI=2,
            output_dir=output_dir,
            rgb_output_dir=rgb_output_dir
        )

In [ ]:

test_lif_ids = [
    7114,7120,7155,7222,
    7229,7239,7284,7293
    ]

base_path = r"C:\Users\lucaz\OneDrive - Politecnico di Milano\Notebooks\università\MAGISTRALE\TESI\MIND_TESI_IMMAGINI"
# Process MSA and Control files
for lif_ids, folder_name in [(test_lif_ids, "MIX_TEST")]:
    print(f"\nProcessing LIF IDs: {lif_ids}")
    tiff4c_folder = os.path.join(base_path, "test_4c")
    tiff3c_rgb_folder = os.path.join(base_path, "test_3cRGB")
    # Determine output directory based on which list we're processing
    
    output_dir = os.path.join(tiff4c_folder, "test")
    rgb_output_dir = os.path.join(tiff3c_rgb_folder, "test3crgb")
    source_folder = "MIX_TEST"  

    print(f"\nProcessing {source_folder} files...")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(rgb_output_dir, exist_ok=True)

    # Create source directory path
    source_dir = os.path.join(base_path, source_folder)
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"Source directory not found: {source_dir}")

    for lif_id in lif_ids:
        lif_path = os.path.join(source_dir, f"{lif_id}.lif")
        if not os.path.exists(lif_path):
            raise FileNotFoundError(f"LIF file not found: {lif_path}")

        process_pairs(
            lif_path,
            channel_to_add_from_DAPI=2,
            output_dir=output_dir,
            rgb_output_dir=rgb_output_dir
        )

In [3]:
import os
import shutil

def organize_images(source_dir, id_list):
    """
    Organize images by copying them to MSA-P or MSA-C subfolders based on ID presence.

    Parameters:
    - source_dir (str): Path to the directory containing images.
    - id_list (list of int): List of IDs to check in image names.
    """
    msa_p_dir = os.path.join(source_dir, "MSA-P")
    msa_c_dir = os.path.join(source_dir, "MSA-C")
    
    # Create subfolders if they don't exist
    os.makedirs(msa_p_dir, exist_ok=True)
    os.makedirs(msa_c_dir, exist_ok=True)
    
    # Convert IDs to strings for matching
    id_str_list = [str(id_) for id_ in id_list]
    
    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)
        
        # Skip directories
        if os.path.isdir(file_path):
            continue
        
        # Check if filename contains any ID from the list
        if any(id_str in filename for id_str in id_str_list):
            target_dir = msa_p_dir
        else:
            target_dir = msa_c_dir
        
        # Copy the file to the target directory
        shutil.copy2(file_path, target_dir)
        print(f"Copied {filename} to {target_dir}")


msap_ids = [
    4121, 5349, 5358, 5435, 5717, 5745, 5753, 
    5767, 5776, 5878, 5978, 5992, 6050, 6053, 
    6179, 6308, 6311, 6326, 6343, 6485, 6593,
    6599, 6657, 7120, 7132, 7144, 7343 
]

input_folder = r"/home/zano/Documents/TESI/3c_MIP_new/MSA"
output_dir = input_folder
# rgb_output_dir = os.path.join(base_path, "18_3cRGB_new", "MSA")   
source_directory = output_dir
organize_images(source_directory, msap_ids)
# source_directory = rgb_output_dir
# organize_images(source_directory, msap_ids)

Copied MAX_4121.lif - 4121 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 2.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-P
Copied MAX_5904 gh2.tif.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-C
Copied MAX_7132.lif - 7132 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh n2.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-P
Copied MAX_5717.lif - 5717 DL VIP r TH b Sinapto gr DAPI grey 63x z2 gh pinhole 1 z 05.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-P
Copied MAX_5969 gh2.tif.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-C
Copied MAX_7105.lif - 7105 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh  n2.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-C
Copied MAX_7144.lif - 7144 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-P
Copied MAX_6046 gh2.tif.tif to /home/zano/Documents/TESI/3c_MIP_new/MSA/MSA-C
Copied MAX_5776 gh2.tif.tif to /home/zano/Documents/

In [ ]:
import os
import re
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import shutil

def parse_name(name):
    """
    Extracts (id_, subject) from a filename using regular expressions.
    Adjust to your actual naming patterns as needed.
    """
    id_match = re.search(r"(\d{4})", name)
    subject_match = re.search(r"\b(gh(?:\s*n?\s*2)?|vaso(?:\s*n?\s*2)?)\b", name, re.IGNORECASE)
    
    if id_match and subject_match:
        id_ = id_match.group(1)
        subject = subject_match.group(1).lower()
        return id_, subject
    return None, None

def group_images_by_id_subject(image_paths):
    """
    Groups images by (id_, subject) into a dictionary.
    
    Args:
        image_paths: List of (full_path, filename).
    
    Returns:
        Dictionary: { (id_, subject): [ (full_path, filename), ... ], (id_2, subject) : [ ... ] }
    1. For each image, extract (id_, subject) using `parse_name`.
    2. If (id_, subject) is found, add the image to the corresponding list in the dictionary.
    3. If (id_, subject) is not found, skip the image.
    4. Return the dictionary.
    """
    series_dict = {}
    for full_path, fname in image_paths:
        id_, subject = parse_name(fname)
        if id_ and subject:
            key = (id_, subject)
            # if key (id,subject) not in series_dict initialize it
            if key not in series_dict:
                series_dict[key] = []
            # append the image to the list of images for that key
            series_dict[key].append((full_path, fname))
    return series_dict

def find_tif_images_in_folder(folder_path):
    """
    Recursively finds all '.tif' or '.tiff' files in `folder_path`.
    Returns a list of (full_path, filename).
    """
    image_paths = []
    # Walk through the directory tree rooted at top = folder_path
    for root, dirs, files in os.walk(top=folder_path):
        # Iterate over each file in the current directory
        for f in files:
            # Check if the file has a .tif or .tiff extension (case-insensitive)
            if f.lower().endswith(('.tif', '.tiff')):
                # Construct the full path to the file
                full_path = os.path.join(root, f)
                # Append the full path and filename to the image_paths list
                image_paths.append((full_path, f))
    # Return the list of image paths
    return image_paths

def filter_keep_th_images(series_dict):
    """
    For each (id_, subject) group, keep only images containing 'TH'
    if more than one image is found. Discard the rest (like 'DAPI').
    
    If no 'TH' image is found in that group, keep them all.
    
    Returns a new dict with the filtered results.
    """
    filtered_dict = {}
    for key, file_list in series_dict.items():
        # If there's only 1 file, we keep it by default
        if len(file_list) == 1:
            filtered_dict[key] = file_list
            continue
        
        # More than one image -> print a message
        print(f"For (id={key[0]}, subject={key[1]}) found {len(file_list)} images. Filtering by 'TH'...")
        
        # Keep only files with 'TH' in their name (case-insensitive)
        th_files = [f for f in file_list if "th" in f[1].lower()]
        
        if th_files:
            filtered_dict[key] = th_files
            print(f"  -> Keeping {len(th_files)} file(s) with 'TH' in the name.")
        else:
            # If no TH images found, keep them all (or decide otherwise).
            filtered_dict[key] = file_list
            print(f"  -> No 'TH' found; keeping all {len(file_list)} file(s).")
    return filtered_dict

# Figure out which axis is channels
def get_num_channels(arr):
    if arr.ndim == 3:
        return arr.shape[-1] if arr.shape[-1] <= 4 else arr.shape[0]
    raise ValueError(f"Cannot determine number of channels from shape: {arr.shape}")
    
def show_channels_and_select_removal(image_array, image_title=""):
    """
    Displays each channel in a separate subplot and prompts the user
    which channel to remove. Returns (ch_to_remove, channel_axis).
    
    Uses color maps in the order: Greens, Blues, gray, Reds (then repeats if needed).
    """
    arr = image_array
    if arr.ndim != 3:
        raise ValueError(f"Expected a 3D array (multi-channel), got shape {arr.shape}")
    
    # Figure out which axis is channels
    if arr.shape[-1] <= 4:  # likely (height, width, channels)
        print("image in the form (H, W, C)")
        channel_axis = -1
        num_channels = arr.shape[-1]
    elif arr.shape[0] <= 4: # possibly (channels, height, width)
        print("image in the form (C, H, W)")
        channel_axis = 0
        num_channels = arr.shape[0]
    else:
        raise ValueError(f"Cannot determine channel axis from shape: {arr.shape}")
    
    # A small color map cycle for up to 4 channels.
    cmap_cycle = ["Greens", "Blues", "gray", "Reds"]
    
    plt.figure(figsize=(12, 4))
    for c in range(num_channels):
        plt.subplot(1, num_channels, c+1)
        if channel_axis == -1:
            channel_image = arr[:, :, c]
        else:
            channel_image = arr[c, :, :]
        cmap = cmap_cycle[c % len(cmap_cycle)]
        plt.imshow(channel_image, cmap=cmap)
        plt.title(f"Channel {c} ({cmap})")
        plt.axis('off')
    plt.suptitle(image_title)
    plt.show(block=True)
    
    # Ask user which channel index to remove
    ch_to_remove = 2#input(f"Enter the channel index [0-{num_channels-1}] to remove for {image_title}: ")
    try:
        ch_to_remove = int(ch_to_remove)
        if ch_to_remove < 0 or ch_to_remove >= num_channels:
            raise ValueError("Index out of range.")
    except ValueError:
        print("Invalid channel index. Skipping removal.")
        return None
    
    return ch_to_remove, channel_axis

def remove_channel_and_save(image_array, ch_to_remove, channel_axis, save_path):
    """
    Removes the specified channel from `image_array` along `channel_axis`
    and saves the resulting 3-channel array to `save_path`.
    """
    
    arr_3ch = np.delete(image_array, ch_to_remove, axis=channel_axis)
    print(f"Removed channel {ch_to_remove} from axis {channel_axis}. New shape: {arr_3ch.shape}")
    tifffile.imwrite(save_path, arr_3ch)
    print(f"Saved 3-channel image to: {save_path}")

def main(folder_path):
    # 1) Find all TIFF images
    all_images = find_tif_images_in_folder(folder_path)
    if not all_images:
        print("No tif images found in the specified folder.")
        return

    # 2) Group by (id_, subject)
    grouped = group_images_by_id_subject(all_images)

    # 3) Filter groups to keep only images with "TH" if there's more than one
    filtered = filter_keep_th_images(grouped)
    
    # Define the output base folder as a sibling of folder_path.
    parent_folder = os.path.dirname(folder_path)
    output_base = os.path.join(parent_folder, "3c_version")
    os.makedirs(output_base, exist_ok=True)
    
    # 4) Process each surviving image
    for key, file_list in filtered.items():
        for (full_path, fname) in file_list:
            # Read the tif
            img = tifffile.imread(full_path)
            shape_ = img.shape  # e.g. (C, Y, X) or (Y, X, C)
            print(f"Processing {fname} with shape {shape_}")
            
            # Determine the original directory and compute its relative path to folder_path.
            original_dir = os.path.dirname(full_path)
            relative_subfolder = os.path.relpath(original_dir, folder_path)
            # If the file is directly under folder_path, use the folder's basename
            if relative_subfolder == ".":
                relative_subfolder = os.path.basename(folder_path)
            # Create the corresponding output directory inside output_base.
            out_dir = os.path.join(output_base, relative_subfolder)
            os.makedirs(out_dir, exist_ok=True)
            
            # CASE A: It's 4-channel => ask user which channel to remove
            if (len(shape_) == 3) and (4 in shape_):
                print(f"\nProcessing 4-channel image: {fname}")
                ch_to_remove, channel_axis = show_channels_and_select_removal(img, image_title=fname)
                
                out_name = os.path.splitext(fname)[0] + "_3ch.tif"
                out_path = os.path.join(out_dir, out_name)
                remove_channel_and_save(img, ch_to_remove, channel_axis, out_path)
            
            # CASE B: It's already 3-channel => if filename doesn't contain "dapi", copy to out_dir
            elif (len(shape_) == 3) and (3 in shape_):
                if "dapi" not in fname.lower():
                    out_name = os.path.splitext(fname)[0] + "_3ch.tif"
                    out_path = os.path.join(out_dir, out_name)
                    tifffile.imwrite(out_path, img)
                    print(f"Copied 3-channel image (no DAPI) to: {out_path}")
                else:
                    print(f"Skipped 3-channel image containing 'dapi': {fname}")
            
            # Otherwise, skip the image
            else:
                print(f"Image '{fname}' is not 4-channel or 3-channel (shape={shape_}); skipping additional processing.")
                
if __name__ == "__main__":
    # Replace below with the path of your main folder containing subfolders with TIFF images
    folder_to_process = r"/home/zano/Documents/TESI/IMAGES_LIF/mix3_4c_MIP"
    main(folder_to_process)

let's fix image ordering
4 channel images are already in the right order which is: 
- green-Blue-Gray-Red

but there are also 3 channel images which are in another order ie.  
- Red-Green-Blue 
- Red-Green-Gray

we have agreed to the standard order Green-Blue-Red for 3channel images and Green-Blue-Gray-Red for 4channel images

We have to make a code which produce:
- filter only glands 
- in the case of 3 channels glands it reorders them to from RGB to GBR 
- in the case of 4 channel do nothing

In [ ]:
import os
import tifffile
import shutil

# Use the folder path defined in the previous cell
folder_path = '/home/zano/Documents/TESI/IMAGES_LIF/control_MIP'  # Or replace with the specific folder path you want to inspect
three_channel_mip_path = os.path.join(folder_path, "3c_version")
four_channel_mip_path = os.path.join(folder_path, "4c_version")

os.makedirs(three_channel_mip_path, exist_ok=True)
os.makedirs(four_channel_mip_path, exist_ok=True)

os.makedirs(folder_path, exist_ok=True)
print(f"Inspecting TIFF files in folder: {folder_path}")

# Check if the folder exists
if not os.path.isdir(folder_path):
    print(f"Error: Folder not found at {folder_path}")
else:
    # Iterate through all files in the specified directory
    for filename in os.listdir(folder_path):
        # Check if the file is a TIFF file
        if filename.lower().endswith(('.tif', '.tiff')):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the TIFF file
                img_array = tifffile.imread(file_path)
                # Print the filename and the shape of the image array
                print(f"File: {filename}, Shape: {img_array.shape}")
                if img_array.shape == (3, 1024, 1024):
                    # Move the file to the 3-channel folder
                    shutil.move(file_path, three_channel_mip_path)
                    print(f"Moved {filename} to {three_channel_mip_path}")
                elif img_array.shape == (4, 1024, 1024):
                    shutil.move(file_path, four_channel_mip_path)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
                


Inspecting TIFF files in folder: /home/zano/Documents/TESI/IMAGES_LIF/control_MIP
File: MAX_6731.lif - 6731 DL VIP r Sinapto gr TH b 63x z2 gh 2 pinhole 1 z 05.tif, Shape: (3, 1024, 1024)
Moved MAX_6731.lif - 6731 DL VIP r Sinapto gr TH b 63x z2 gh 2 pinhole 1 z 05.tif to /home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version
File: MAX_5168.lif - 5168 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 3.tif, Shape: (4, 1024, 1024)
File: MAX_5167.lif - 5167 DL VIP red Sinapto gr TH b D grey 63x z2 pinhole 1 z 05 gh.tif, Shape: (4, 1024, 1024)
File: MAX_5199.lif - 5199 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif, Shape: (4, 1024, 1024)
File: MAX_6743.lif - 6743 DL VIP r Sinapto gr TH b 63x z2 gh 2 pinhole 1 z 05.tif, Shape: (4, 1024, 1024)
File: MAX_5197.lif - 5197 gh DL VIP r TH b Sinapto gr 63x z2 pinhole 06 z 022.tif, Shape: (4, 1024, 1024)
File: MAX_5167.lif - 5167 DL VIP red Sinapto gr TH b D grey 63x z2 pinhole 1 z 05 gh2.tif, Shape: (4, 1024, 1024

In [8]:
%pip install ome-types


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.1/224.1 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np 
import tifffile
import os

def remove_channel(file_path,channel_to_remove = 3) -> np.ndarray:
    img_array = tifffile.imread(file_path)
    print(isinstance(img_array, np.ndarray))
    new_img_array = np.delete(img_array, channel_to_remove, axis=0)
    return new_img_array
    


# code to remove the dapi channel from 4channel images reducing them to 3channel images
source_folder = r"/home/zano/Documents/TESI/IMAGES_LIF/control_MIP/4c_version"
output_folder = r"/home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version"
os.makedirs(output_folder, exist_ok=True)
dapi_channel_number = 2
# Iterate through all files in the specified directory
for filename in os.listdir(source_folder):
    # Check if the file is a TIFF file
    if filename.lower().endswith(('.tif', '.tiff')):
        file_path = os.path.join(source_folder, filename)
        try:
            three_c_array = remove_channel(file_path,channel_to_remove = dapi_channel_number)
            three_c_array = np.expand_dims(three_c_array, axis=0)  #ADD the T dim to  Ensure data is (1, C, Y, X) to match OME-TIFF format (T, C, Y, X)
            save_path = os.path.join(output_folder, filename)
            tifffile.imwrite(
                save_path,
                three_c_array,
                metadata={'axes': 'TCYX'},
                photometric='minisblack',
                ome=True
            )
            
            print(f"Saved {filename} to {save_path}")
        except Exception as e:
            print(f"Error reading file {filename}: {e}")
    

True
Saved MAX_5168.lif - 5168 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 3.tif to /home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version/MAX_5168.lif - 5168 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 3.tif
True
Saved MAX_5167.lif - 5167 DL VIP red Sinapto gr TH b D grey 63x z2 pinhole 1 z 05 gh.tif to /home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version/MAX_5167.lif - 5167 DL VIP red Sinapto gr TH b D grey 63x z2 pinhole 1 z 05 gh.tif
True
Saved MAX_5199.lif - 5199 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif to /home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version/MAX_5199.lif - 5199 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif
True
Saved MAX_6743.lif - 6743 DL VIP r Sinapto gr TH b 63x z2 gh 2 pinhole 1 z 05.tif to /home/zano/Documents/TESI/IMAGES_LIF/control_MIP/3c_version/MAX_6743.lif - 6743 DL VIP r Sinapto gr TH b 63x z2 gh 2 pinhole 1 z 05.tif
True
Saved MAX_5197.lif - 5197 gh DL VIP r TH 

# TIFF TO PNG FROM A FOLDER

In [9]:
#!/usr/bin/env python3
"""
Batch-convert 4-channel TIFFs (C, H, W) → 4-channel PNGs (R, G, B, A).
Author: <you>; 2025-05-29
"""
from pathlib import Path
import select
import tifffile          as tiff
import imageio.v3        as iio    # v3 API gives ndarray IO
import numpy             as np

def reorder_channels(arr: np.ndarray) -> np.ndarray:
    """
    Rearrange channels so they become RGB or RGBA as requested.

    Input order assumptions
    -----------------------
    * 3-channel TIFF :  (G, B, R)                        → (R, G, B)
    * 4-channel TIFF :  (G, B, Gray, R)  (=GBλR)         → (R, G, Gray, B)

      # index : 0    1     2      3
      # new   : 3    0     2      1

    Raises
    ------
    ValueError
        If the TIFF has anything other than 3 or 4 planes.
    """
    if arr.ndim != 3:
        raise ValueError(f"Expected (C, H, W); got {arr.shape}")

    c = arr.shape[0]
    if c == 3:          # GBR  →  RGB
        return arr[[2, 0, 1], :, :]
    if c == 4:          # GBλR → R G λ B
        return arr[[3, 0, 2, 1], :, :]

    raise ValueError(f"Unsupported channel count: {c}")

def compose_png(arr: np.ndarray) -> np.ndarray:
    """
    Expects a (4, H, W) array in float or uint-type.
    Returns a uint8 (H, W, 4)   suitable for PNG-RGBA.
    it normalises the input array to 0-255 range and rearranges channels.
    Modify here if you want to recolour channels, normalise, etc.
    """
    assert arr.shape[0] == 4 or arr.shape[0] == 3, "Need exactly 3 or 4 source channels"
    # -------- optional normalisation (0-255) ----------
    if arr.dtype != np.uint8:
        arr = arr.astype(np.float32)
        arr = 255.0 * (arr - arr.min()) / (arr.ptp() + 1e-8)
    arr = arr.astype(np.uint8)
    # 2. rearrange channels
    arr = reorder_channels(arr)
    # TIFF = (C, H, W); PNG wants (H, W, C)
    return np.transpose(arr, (1, 2, 0))


def tiffs_to_pngs(src_dir: Path, dst_dir: Path) -> None:
    dst_dir.mkdir(parents=True, exist_ok=True)
    for tif_path in src_dir.glob("*.tif*"):
        img = tiff.imread(tif_path)
        png = compose_png(img)
        out_path = dst_dir / (tif_path.stem + ".png")
        iio.imwrite(out_path, png, plugin="pillow")   # pillow handles RGBA
        print(f"✔  {tif_path.name}  →  {out_path.name}")


if __name__ == "__main__":
    #patient_type =
    num_channels = int(input("Enter the number of channels (3 or 4): "))
    if num_channels not in [3, 4]:
        raise ValueError("Invalid number of channels. Please enter 3 or 4.")
    # Define available patient types and their corresponding folder paths
    if num_channels == 3:
        patient_types = {
            "MSA": "/home/zano/Documents/TESI/3c_MIP_new/MSA",
            "PD": "/home/zano/Documents/TESI/3c_MIP_new/PD",
            "MSA-P": "/home/zano/Documents/TESI/3c_MIP_new/MSA-P"
        }
    elif num_channels == 4:
        patient_types = {
            "MSA": "/home/zano/Documents/TESI/4c_MIP/MSA",
            "PD": "/home/zano/Documents/TESI/4c_MIP/PD",
            "MSA-P": "/home/zano/Documents/TESI/4c_MIP/MSA-P"
        }
    
    # Prompt the user to select a patient type
    print("Available patient types:")
    for key in patient_types.keys():
        print(f"- {key}")
    
    selected_type = input("Enter the patient type: ").upper()
    
    # Validate the user input
    if selected_type not in patient_types:
        raise ValueError("Invalid patient type selected.")
    
    # Get the source folder path based on the selected patient type
    SRC = Path(patient_types[selected_type])
    
    # Define the destination folder path based on the selected patient type
    if num_channels == 3:
        DST = Path(f"/home/zano/Documents/TESI/3c_MIP_new/{selected_type}_PNG")
    elif num_channels == 4:
        DST = Path(f"/home/zano/Documents/TESI/4c_MIP/{selected_type}_PNG")

    # Process the TIFF files
    tiffs_to_pngs(SRC, DST)

Available patient types:
- MSA
- PD
- MSA-P
✔  MAX_6791.lif - 6791 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 2.tif  →  MAX_6791.lif - 6791 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh 2.png
✔  MAX_6375.lif - 6375 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif  →  MAX_6375.lif - 6375 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.png
✔  MAX_6616.lif - 6616 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif  →  MAX_6616.lif - 6616 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.png
✔  MAX_7155.lif - 7155 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif  →  MAX_7155.lif - 7155 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.png
✔  MAX_6366.lif - 6366 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.tif  →  MAX_6366.lif - 6366 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 05 gh.png
✔  MAX_6651.lif - 6651 DL VIP red Sinapto gr TH b D grey 63x z 2 pinhole 1 z 

# CHECK IMAGES SHAPES INSIDE A FOLDER

In [3]:
!pip install tifffile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 6.5 MB/s eta 0:00:0000:01


In [7]:
!cd ..

In [15]:
import os
import tifffile
from collections import Counter
import pandas as pd

def check_tiff_shapes(folder_path):
    """
    Check the shapes of all TIFF images in the specified folder.
    
    Args:
        folder_path (str): Path to the folder containing TIFF images
        
    Returns:
        pd.DataFrame: DataFrame with image filenames and their shapes
    """
    # Verify the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder not found at {folder_path}")
        return None
    
    # List to store results
    results = []
    shapes_counter = Counter()
    
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.tif', '.tiff')):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the image and get its shape
                img = tifffile.imread(file_path)
                shape = img.shape
                shapes_counter[shape] += 1
                
                # Add to results
                results.append({
                    'filename': filename,
                    'shape': shape,
                    'dimensions': len(shape),
                    'min_value': img.min(),
                    'max_value': img.max(),
                    'dtype': img.dtype
                })
                
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                results.append({
                    'filename': filename,
                    'shape': 'ERROR',
                    'dimensions': 'ERROR',
                    'min_value': 'ERROR',
                    'max_value': 'ERROR',
                    'dtype': 'ERROR'
                })
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"Found {len(results)} TIFF files in {folder_path}")
    print("\nShape distribution:")
    for shape, count in shapes_counter.most_common():
        print(f"  {shape}: {count} files")
    
    return df

# Example usage:
folder_to_check = "/home/zano/Documents/TESI/FOLDER_CINECA/data/PRETRAINING_MSA_VS_PD"
df = check_tiff_shapes(folder_to_check)
display(df)

Found 127 TIFF files in /home/zano/Documents/TESI/FOLDER_CINECA/data/PRETRAINING_MSA_VS_PD

Shape distribution:
  (2, 1024, 1024): 80 files
  (3, 1024, 1024): 47 files


,filename,shape,dimensions,min_value,max_value,dtype
0,MAX_5349.lif - 5349 DL VIP red Sinapto gr TH b...,"(3, 1024, 1024)",3,0,255,uint8
1,MAX_6006 6007 6008.lif - 6008 DL PGP gr Psyn r...,"(2, 1024, 1024)",3,0,25,uint8
2,MAX_5767.lif - 5767 DL VIP r TH b Sinapto gr D...,"(3, 1024, 1024)",3,0,255,uint8
3,MAX_5878.lif - 5878 DL VIP r TH b Sinapto gr 6...,"(2, 1024, 1024)",3,0,29,uint8
4,MAX_6731.lif - 6731 DL VIP r Sinapto gr TH b 6...,"(3, 1024, 1024)",3,0,255,uint8
...,...,...,...,...,...,...
122,MAX_5168.lif - 5168 DL VIP red Sinapto gr TH b...,"(3, 1024, 1024)",3,0,255,uint8
123,MAX_6363.lif - 6363 DL VIP red Sinapto gr TH b...,"(3, 1024, 1024)",3,0,255,uint8
124,MAX_6177 6178 6179.lif - Ev DL 63x gh.tif,"(2, 1024, 1024)",3,0,245,uint8
125,MAX_6183 6184 6185.lif - 6184 Pth PGP gr Psyn ...,"(2, 1024, 1024)",3,0,255,uint8


In [19]:
import os
import tifffile
import shutil
from pathlib import Path

def remove_images_with_few_channels(folder_path, min_channels=3, backup_folder=None, dry_run=False):
    """
    Remove or move images that have fewer than the specified minimum number of channels.
    
    Args:
        folder_path (str): Path to the folder containing images
        min_channels (int): Minimum number of channels required (default: 3)
        backup_folder (str, optional): Path to move removed files instead of deleting them
        dry_run (bool): If True, only prints what would be removed without actually doing it
        
    Returns:
        dict: Summary of removed files and their shapes
    """
    folder_path = Path(folder_path)
    
    if not folder_path.exists():
        print(f"Error: Folder not found at {folder_path}")
        return None
    
    # Create backup folder if specified
    if backup_folder and not dry_run:
        backup_path = Path(backup_folder)
        backup_path.mkdir(parents=True, exist_ok=True)
    
    removed_files = []
    kept_files = []
    error_files = []
    
    # Iterate through all TIFF files
    for file_path in folder_path.glob("*.tif*"):
        try:
            # Read image and check shape
            img = tifffile.imread(file_path)
            
            # Determine number of channels based on shape
            if img.ndim == 2:
                num_channels = 1
            elif img.ndim == 3:
                # Could be (H, W, C) or (C, H, W)
                # Assume channels is the smallest dimension if <= 4
                if min(img.shape) <= 4:
                    num_channels = min(img.shape)
                else:
                    # If all dimensions > 4, assume last dimension is channels
                    num_channels = img.shape[-1] if img.shape[-1] <= 4 else 1
            else:
                # For 4D arrays, assume format is (Z, C, H, W) or (Z, H, W, C)
                if img.ndim == 4:
                    # Find dimension with size <= 4 (likely channels)
                    channel_candidates = [i for i, size in enumerate(img.shape) if size <= 4]
                    if channel_candidates:
                        num_channels = img.shape[channel_candidates[0]]
                    else:
                        num_channels = 1
                else:
                    num_channels = 1
            
            # Check if image should be removed
            if num_channels < min_channels:
                removed_files.append({
                    'filename': file_path.name,
                    'shape': img.shape,
                    'channels': num_channels
                })
                
                if not dry_run:
                    if backup_folder:
                        # Move to backup folder
                        backup_file = backup_path / file_path.name
                        shutil.move(str(file_path), str(backup_file))
                        print(f"Moved {file_path.name} (shape: {img.shape}, channels: {num_channels}) to backup folder")
                    else:
                        # Delete the file
                        file_path.unlink()
                        print(f"Deleted {file_path.name} (shape: {img.shape}, channels: {num_channels})")
                else:
                    print(f"Would remove {file_path.name} (shape: {img.shape}, channels: {num_channels})")
            else:
                kept_files.append({
                    'filename': file_path.name,
                    'shape': img.shape,
                    'channels': num_channels
                })
                
        except Exception as e:
            error_files.append({
                'filename': file_path.name,
                'error': str(e)
            })
            print(f"Error processing {file_path.name}: {e}")
    
    # Print summary
    print(f"\nSummary:")
    print(f"Total files processed: {len(removed_files) + len(kept_files) + len(error_files)}")
    print(f"Files with < {min_channels} channels: {len(removed_files)}")
    print(f"Files kept: {len(kept_files)}")
    print(f"Files with errors: {len(error_files)}")
    
    if dry_run:
        print("\n*** DRY RUN - No files were actually removed ***")
    
    return {
        'removed': removed_files,
        'kept': kept_files,
        'errors': error_files
    }

# Example usage:
# First, do a dry run to see what would be removed
folder_to_clean = "/home/zano/Documents/TESI/FOLDER_CINECA/data/PRETRAINING_MSAP_VS_PD"
backup_folder = "/home/zano/Documents/TESI/FOLDER_CINECA/data/backup_removed_files"

print("=== DRY RUN ===")
results = remove_images_with_few_channels(
    folder_path=folder_to_clean,
    min_channels=3,
    backup_folder=backup_folder,
    dry_run=True
)

# If you're satisfied with the dry run results, run it for real:
print("\n=== ACTUAL REMOVAL ===")
results = remove_images_with_few_channels(
    folder_path=folder_to_clean,
    min_channels=3,
    backup_folder=backup_folder,
    dry_run=False
)

=== DRY RUN ===
Would remove MAX_6006 6007 6008.lif - 6008 DL PGP gr Psyn red 20 x z2 nervi e gh.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_5878.lif - 5878 DL VIP r TH b Sinapto gr 63x z2 vaso 2 pinhole 1 z 05.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6026 6027 6028.lif - 6026 C7 PGP gr Psyn red 20 x gh.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6183 6184 6185.lif - 6185  DL PGP gr Psyn red 20x gh sez2.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6006 6007 6008.lif - 6006 C7 PGP gr Psyn red 20 x z2 gh.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6041 6042 6043 .lif - 6043  v2  DL gh PGP gr P SYN red 20x.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6550 6552 6554.lif - 6550 gh PGP gr P SYN red 20x z3.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6183 6184 6185.lif - 6184 Pth PGP gr Psyn red 20x MEP e gh 2.tif (shape: (2, 1024, 1024), channels: 2)
Would remove MAX_6177 6178 6179.lif - 

In [11]:
!cd /home/zano/Documents/TESI/FOLDER_CINECA/